In [1]:

%matplotlib inline
import os
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import SimpleRNN

from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import math
from sklearn.metrics import mean_squared_error
import FinanceDataReader as fdr

Using TensorFlow backend.


In [2]:
#특성 추가 ------------------------------------------
    #이동평균선
def get_MA(df):
    MA_26=df["Close"].rolling(26).mean()
    MA_52=df["Close"].rolling(52).mean()
    df=df.assign(MA_26=MA_26,MA_52=MA_52).dropna()
    
    return df

    
    #스토캐스틱
def get_stochastic(df, n=15, m=5, t=3):
    # n일중 최고가
    ndays_high = df.High.rolling(window=n, min_periods=1).max()
    # n일중 최저가
    ndays_low = df.Low.rolling(window=n, min_periods=1).min()
 
    # Fast%K 계산
    kdj_k = ((df.Close - ndays_low) / (ndays_high - ndays_low))*100
    # Fast%D (=Slow%K) 계산
    kdj_d = kdj_k.ewm(span=m).mean()
    # Slow%D 계산
    kdj_j = kdj_d.ewm(span=t).mean()
 
    # dataframe에 컬럼 추가
    df = df.assign(kdj_k=kdj_k, kdj_d=kdj_d, kdj_j=kdj_j).dropna()
    
    return df
   
    #시간
def get_time(df):
    time=np.linspace(0,10,len(df),endpoint=False).reshape(-1,1)
    df=df.assign(time=time)
    
    return df

NameError: name 'pandf' is not defined

In [9]:
KS11 = fdr.DataReader("KS11","2015-01-01","2019-01-01")
KQ11= fdr.DataReader("KQ11","2015-01-01","2019-01-01")
US500 = fdr.DataReader("US500","2015-01-01","2019-01-01")
HSI = fdr.DataReader("HSI","2015-01-01","2019-01-01")
IXIC = fdr.DataReader("IXIC","2015-01-01","2019-01-01")

In [15]:
df

NameError: name 'df' is not defined

In [17]:
import time
start = time.time()
Xtrain_size=0

END=0
초기돈=10000
d=1
dd=19
BATCH = 20
투자결과=[]
총df=pd.DataFrame(columns=["영업일","인덱스","예측값","실제수익률"])
세트df=pd.DataFrame(columns=["영업일","인덱스","예측값","실제수익률"])

for x in range(int(min(len(KS11),len(KQ11),len(US500),len(HSI),len(IXIC))/20)): #예측머신돌리기
     
    if END==1:
        break
        
        
    if (x==0): #day만들기
        day= str(d) + "~" + str(dd) + "일 차" 
    else:
        day= str(d+(20*x)) +"~" + str(dd+(20*x)) + "일 차"
    print(day,"---------------------------------")
    
    세트df=pd.DataFrame(columns=["영업일","인덱스","예측값","실제수익률"])
    
    for 인덱스,인덱스명 in zip([KS11,KQ11,IXIC,HSI,US500],["KS11","KQ11","IXIC","HSI","US500"]):

        #종료 조건
        if Xtrain_size==len(인덱스)-BATCH:
            print(" 종료 --------------")
            print(총df)
            print("time : ",time.time() - start)
            END=1
            break
    
            
        
        
        
        #역사적변동성 추가 ---------------------------------
        
        import math
        list=[] #로그수익률
        for xx in range(인덱스.shape[0]-1):
            list.append(math.log(인덱스["Close"][xx+1] / 인덱스["Close"][xx]))
        average = sum(list)/len(list) # 로그수익률의 평균

        list2=[] # 로그수익률의 편차의 제곱
        for xx in range(인덱스.shape[0]-1):
            list2.append((average - (math.log(인덱스["Close"][xx+1] / 인덱스["Close"][xx]))) **2)
        average2 = sum(list2)/len(list2) #편차의 제곱의 평균

        V=math.sqrt(average2 * 252) # 역사적변동성

        list3=[] # 그냥 표준편차
        for xx in 인덱스["Change"]:
            xx=abs(xx)
            list3.append(math.sqrt(xx)/V) # 표준편차/역사적변동성
    
        인덱스 = 인덱스.assign(V=list3)
    
    
    
    
        # feature 추가
        
        
        df = 인덱스
        df=get_MA(df)
        df=get_stochastic(df)
        df=get_time(df)
        df=df[0:-(len(df)%BATCH)]
        #-----------------------------------------------
    
    
    
    
    
    
    
        
        #예측머신--------------------------------------------------------

    
        
        
        MA_26=df["Close"].rolling(21).mean()
        y_before=MA_26.dropna()
        df=df[20:]
        
        y=[]
        for a in range(1,10001):
            try:
                y.append(np.where(df["Close"][-1+(BATCH*a)]>y_before[-1+(BATCH*a)],1,0)) # 이평선비교
                #y.append(np.where(df["Close"][-1+(BATCH*a)] > df["Close"][-1+(BATCH*(a-1))],1,0)) #값비교
            except:
                break

        df = df.values
        df.astype('float32')
        scaler = MinMaxScaler(feature_range=(0, 1))
        df = scaler.fit_transform(df)

        Xtrain_size=int(df.shape[0]/BATCH*0.7)*BATCH + (x*BATCH)

        X_train=df[0:Xtrain_size]
        X_test=df[Xtrain_size:Xtrain_size+BATCH]

        y_train=y[ 0 : int(Xtrain_size/BATCH) ]
        y_test=y[int(Xtrain_size/BATCH)]

        # many to one 전처리 필요
        y_train = np.reshape(y_train, (len(y_train), 1))
        y_test = np.reshape(y_test, (1, 1))
        X_train = np.reshape(X_train, (int(X_train.shape[0]/BATCH),BATCH, X_train.shape[1]))
        X_test = np.reshape(X_test, (int(X_test.shape[0]/BATCH),BATCH, X_test.shape[1]))
        
    
        X_train[:,BATCH-1,:]=0 #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
        X_test[:,BATCH-1,:]=0 #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

        
        # simple lstm network learning
        model = Sequential()
        model.add(LSTM(36, input_shape=(BATCH, 13)))
        for i in range(3):
            model.add(Dense(36,activation='sigmoid'))
        model.add(Dense(1,activation='sigmoid'))
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
        model.fit(X_train, y_train, epochs=10, batch_size=16,validation_split=0.1,verbose=0)

    
        #예측머신 끝 --------predict -> 0 or 1 나옴 ---------------------

        예측 = model.predict(X_test)
        print(예측)
        if 예측>=0.5:
            예측=1
        else:
            예측=0
        
        y_1=인덱스.loc[::19,:]["Close"][:-1] # 1일차,20,40, ...
        실제수익률 = (y_1[x+1]-y_1[x])/ y_1[x]                                         
        턴df=pd.DataFrame(data={"영업일":[day],"인덱스":[인덱스명],"예측값":[예측],"실제수익률":[실제수익률] }
                         , columns=["영업일","인덱스","예측값","실제수익률"])
        
        세트df=pd.concat([세트df,턴df])
        
        if int(턴df["예측값"])==1:
            print("상승 예측된 인덱스 : ",인덱스명)
        
    earn_list=[]
    for x,y in zip(세트df["예측값"],세트df["실제수익률"]):
        if x==1:
            earn_list.append(y)
        
    earn_list2=[]
    
    if len(earn_list)!=0:
        초기돈=초기돈/(len(earn_list))
        for y in earn_list:
            earn_list2.append(초기돈+(초기돈*y))
        초기돈=sum(earn_list2)    
    
    else:
        earn_list2.append(초기돈)
        
    총df=pd.concat([총df,세트df])
    총df = 총df.set_index("영업일")

    print("분산 투자 : ",earn_list2)
    print("투자 결과 : ",초기돈)
    
    print("\n★")
    print(세트df)
    print("★\n")
    
    

        #MONEY = 투잣돈 * A의 20일후 change + 투잣돈 * B의 20일후 change #"20일차MONEY"출력


1~19일 차 ---------------------------------
[[0.6133586]]
상승 예측된 인덱스 :  KS11
[[0.6126909]]
상승 예측된 인덱스 :  KQ11
[[0.67619604]]
상승 예측된 인덱스 :  IXIC
[[0.41300905]]
[[0.651497]]
상승 예측된 인덱스 :  US500
분산 투자 :  [2531.898216399161, 2677.839380203348, 2451.5688170245894, 2423.221747157711]
투자 결과 :  10084.528160784808

★
       영업일    인덱스 예측값     실제수익률
0  1~19일 차   KS11   1  0.012759
0  1~19일 차   KQ11   1  0.071136
0  1~19일 차   IXIC   1 -0.019372
0  1~19일 차    HSI   0  0.030935
0  1~19일 차  US500   1 -0.030711
★

21~39일 차 ---------------------------------
[[0.61333483]]
상승 예측된 인덱스 :  KS11
[[0.6807959]]
상승 예측된 인덱스 :  KQ11
[[0.7100765]]
상승 예측된 인덱스 :  IXIC
[[0.43008578]]
[[0.6479705]]
상승 예측된 인덱스 :  US500
분산 투자 :  [2580.3024413815224, 2643.082536273267, 2699.690742113689, 2659.5232951508065]
투자 결과 :  10582.599014919284

★
        영업일    인덱스 예측값     실제수익률
0  21~39일 차   KS11   1  0.023470
0  21~39일 차   KQ11   1  0.048371
0  21~39일 차   IXIC   1  0.070825
0  21~39일 차    HSI   0  0.009247
0  21~39일 차  US500   

C:\Program Files\anaconda3\lib\site-packages\ipykernel_launcher.py:170: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



[[0.59749347]]
상승 예측된 인덱스 :  KS11
[[0.6384713]]
상승 예측된 인덱스 :  KQ11
[[0.6983001]]
상승 예측된 인덱스 :  IXIC
[[0.4457728]]
[[0.651283]]
상승 예측된 인덱스 :  US500
분산 투자 :  [2676.1100818723326, 2723.554333892239, 2592.257362461688, 2584.867066349048]
투자 결과 :  10576.788844575307

★
        영업일    인덱스 예측값     실제수익률
0  41~59일 차   KS11   1  0.011513
0  41~59일 차   KQ11   1  0.029446
0  41~59일 차   IXIC   1 -0.020181
0  41~59일 차    HSI   0 -0.013141
0  41~59일 차  US500   1 -0.022975
★

61~79일 차 ---------------------------------


C:\Program Files\anaconda3\lib\site-packages\ipykernel_launcher.py:170: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



[[0.6161331]]
상승 예측된 인덱스 :  KS11
[[0.61827415]]
상승 예측된 인덱스 :  KQ11
[[0.5406965]]
상승 예측된 인덱스 :  IXIC
[[0.45844078]]
[[0.5453494]]
상승 예측된 인덱스 :  US500
분산 투자 :  [2845.293920522876, 2860.485041512337, 2748.9677406928254, 2717.215968359373]
투자 결과 :  11171.96267108741

★
        영업일    인덱스 예측값     실제수익률
0  61~79일 차   KS11   1  0.076052
0  61~79일 차   KQ11   1  0.081797
0  61~79일 차   IXIC   1  0.039623
0  61~79일 차    HSI   0  0.160693
0  61~79일 차  US500   1  0.027615
★

81~99일 차 ---------------------------------


...

[[0.5783892]]
상승 예측된 인덱스 :  KS11
[[0.673588]]
상승 예측된 인덱스 :  KQ11
[[0.71596]]
상승 예측된 인덱스 :  IXIC
[[0.4845609]]
[[0.6043203]]
상승 예측된 인덱스 :  US500
분산 투자 :  [2614.366331258092, 2647.3999843893307, 2637.041469066841, 2641.0070681551406]
투자 결과 :  10539.814852869404

★
          영업일    인덱스 예측값     실제수익률
0  241~259일 차   KS11   1 -0.023879
0  241~259일 차   KQ11   1 -0.011545
0  241~259일 차   IXIC   1 -0.015413
0  241~259일 차    HSI   0 -0.040870
0  241~259일 차  US500   1 -0.013932
★

261~279일 차 ---------------------------------


C:\Program Files\anaconda3\lib\site-packages\ipykernel_launcher.py:170: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



[[0.6112968]]
상승 예측된 인덱스 :  KS11
[[0.649971]]
상승 예측된 인덱스 :  KQ11
[[0.7434141]]
상승 예측된 인덱스 :  IXIC
[[0.5001175]]
상승 예측된 인덱스 :  HSI
[[0.71057403]]
상승 예측된 인덱스 :  US500
분산 투자 :  [2049.5275642841552, 2104.688345016896, 1886.6650429947122, 1945.4697598987796, 1919.8613932162798]
투자 결과 :  9906.212105410823

★
          영업일    인덱스 예측값     실제수익률
0  261~279일 차   KS11   1 -0.027721
0  261~279일 차   KQ11   1 -0.001553
0  261~279일 차   IXIC   1 -0.104982
0  261~279일 차    HSI   1 -0.077085
0  261~279일 차  US500   1 -0.089234
★

281~299일 차 ---------------------------------


C:\Program Files\anaconda3\lib\site-packages\ipykernel_launcher.py:170: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



IndexError: list index out of range

In [14]:
# 면역전략의 수익
list=[]
for 인덱스,인덱스명 in zip([KS11,KQ11,IXIC,HSI,US500],["KS11","KQ11","IXIC","HSI","US500"]):
    돈 = 10000
    df=인덱스
    print(인덱스명," : " , 돈 + ( 돈 * (df["Close"][-1] - df["Close"][0])/ df["Close"][0] ))
    print("수익률 : ",((df["Close"][-1] - df["Close"][0])/ df["Close"][0] ) *100 , "%\n")
    list.append((df["Close"][-1] - df["Close"][0])/ df["Close"][0] )

print("분산 투자시")
list2=[]
for x in list:
    print(2000+(2000*x))
    list2.append(2000+(2000*x))
print("\n총 수익 : ",sum(list2))

KS11  :  12033.440635796573
수익률 :  20.334406357965726 %

KQ11  :  12794.220682080704
수익률 :  27.942206820807048 %

IXIC  :  28743.816116651215
수익률 :  187.43816116651212 %

HSI  :  11843.178477295805
수익률 :  18.431784772958064 %

US500  :  22125.967572529324
수익률 :  121.25967572529323 %

분산 투자시
2406.6881271593147
2558.844136416141
5748.763223330243
2368.635695459161
4425.193514505865
총 수익률 :  17508.124696870727


In [45]:
X_train = np.reshape(X_train, (int(X_train.shape[0]/BATCH),BATCH, X_train.shape[1]))[:,:19,:]

ValueError: cannot reshape array of size 20880 into shape (4,20,20)

In [19]:
y_train.shape

(84, 1)